Creating Match Projections using OOP

In [1]:
import pandas as pd
import re

from data_preparation import load_match_summary
from data_preparation import load_player_stats
from data_preparation import aggregate_player_to_match_stats
from data_preparation import load_team_info
from data_preparation import load_venue_info
from data_preparation import create_match_summary_stats

from player import Player
from match import Match
from team import Team
from rating_calculator import PlayerRatingCalculator, TeamRatingCalculator
from projector import Projector

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)
%load_ext autoreload
%autoreload 2

1. load expected vaep data and player data
2. simulate initial updating process
3. put into a function
4. create objective function
5. run optuna optimisation to find update parameters

Load in player and match stats

In [2]:
match_summary = load_match_summary()
player_stats = load_player_stats()
match_stats = aggregate_player_to_match_stats(player_stats)
team_info, home_team_info, away_team_info = load_team_info()
venue_info, away_venue_info = load_venue_info()
match_summary_stats = create_match_summary_stats(match_summary, match_stats, home_team_info, away_team_info, away_venue_info)

Updating Process:

1. Sum up player values from each game as the actual offensive rating for that team
2. Get difference in match expected vaep values as the actual "Team Rating" for that match
3. Take into account home advantage for home team
4. Update long term player ratings
5. Update long term team ratings
5. Calculate "Defensive Rating" as the "Offensive Rating" + "Team Rating" (just for completeness)

Calculate Home Advantage (Rough Overall Average)

In [3]:
overall_ha = (match_summary_stats['Home_exp_vaep_value'].mean() - match_summary_stats['Away_exp_vaep_value'].mean()).round(2)
overall_ha

5.83

## Initialise Teams and Players

In [4]:
DarcyMoore = Player('Darcy Moore', 'Collingwood')
DarcyMoore.rating

{'Initial': 0}

In [5]:
def create_team(team_name, player_stats):
    
    players_list = list(player_stats[player_stats['Team'] == team_name]['Player'].unique())
    player_dict = {}
    for player in players_list:
        player_dict[player] = Player(player, team_name)
    
    return Team(team_name, player_dict)

In [6]:
def initialise_teams_players():
    
    team_list = list(team_info['Team'])
    team_dict = {}
    
    for team in team_list:
        team_dict[team] = create_team(team, player_stats)
    
    for team in team_dict.keys():
    
        team_dict[team].add_rating('2020F4', 0)
        team_dict[team].add_offensive_rating('2020F4', 0)
        team_dict[team].add_defensive_rating('2020F4', 0)
        
        for player in team_dict[team].players.keys():
            
            team_dict[team].players[player].add_rating('2020F4', 0)
            
    
    return team_dict

In [7]:
team_dict = initialise_teams_players()

In [8]:
round_dict = {}

## 202101 

In [9]:
round_id = '202101'

Get player stats for matches based on Match ID

In [10]:
def get_round_match_ids(player_stats, round_id):
    
    round_player_stats = player_stats[player_stats['Round_ID'] == round_id]
    
    return list(round_player_stats['Match_ID'].unique())
    

In [11]:
def get_teams(match_id):
    
    home_team = re.sub(r"(?<=\w)([A-Z])", r" \1", match_id.split("_")[1])
    away_team = re.sub(r"(?<=\w)([A-Z])", r" \1", match_id.split("_")[-1])
    
    return home_team, away_team

Create Match objects

In [12]:
def create_round_matches(player_stats, round_dict, round_id):
    
    round_match_id_list = get_round_match_ids(player_stats, round_id)

    round_dict[round_id] = {}
    for match_id in round_match_id_list:
        home_team, away_team = get_teams(match_id)
        
        round_dict[round_id][match_id] = Match(match_id=match_id,
                                               match_summary=match_summary,
                                               player_stats=player_stats,
                                               home_team=team_dict[home_team],
                                               away_team=team_dict[away_team]
                                               )
        
    return round_dict

In [13]:
round_dict = create_round_matches(player_stats, round_dict, round_id)
round_dict

{'202101': {'202101_BrisbaneLions_Sydney': <match.Match at 0x7ff4d830bf40>,
  '202101_Collingwood_WesternBulldogs': <match.Match at 0x7ff4e9967b80>,
  '202101_Essendon_Hawthorn': <match.Match at 0x7ff4d830b5b0>,
  '202101_GreaterWesternSydney_StKilda': <match.Match at 0x7ff4e99673a0>,
  '202101_Melbourne_Fremantle': <match.Match at 0x7ff4eacd79d0>,
  '202101_NorthMelbourne_PortAdelaide': <match.Match at 0x7ff4eacd71c0>,
  '202101_Richmond_Carlton': <match.Match at 0x7ff4eacd7e80>,
  '202101_WestCoast_GoldCoast': <match.Match at 0x7ff4eacdd0d0>}}

Testing Team Rating Updates

In [34]:
def update_team_ratings(Match, team_dict):
    
    home_squad_list = Match.get_home_match_squad()
    away_squad_list = Match.get_away_match_squad()
    match_projector = Projector(Match, home_squad_list, away_squad_list)
    
    home_rating_calculator = TeamRatingCalculator(team_dict[Match.home_team.get_team_name()])
    home_rating_calculator.update_team_rating(Match, match_projector)
    
    away_rating_calculator = TeamRatingCalculator(team_dict[Match.away_team.get_team_name()])
    away_rating_calculator.update_team_rating(Match, match_projector)
    

In [27]:
for match_id in round_dict[round_id].keys():
    update_team_ratings(round_dict[round_id][match_id], team_dict)

In [28]:
team_dict['Adelaide'].add_rating('202101', 0)
team_dict['Geelong'].add_rating('202101', 0)

In [29]:
for team in team_dict.keys():
    print(team, team_dict[team].rating)

Adelaide {'Initial': 0, '2020F4': 0, '202101': 0}
Brisbane Lions {'Initial': 0, '2020F4': 0, '202101': -15.401882954120005}
Carlton {'Initial': 0, '2020F4': 0, '202101': -23.770569458980034}
Collingwood {'Initial': 0, '2020F4': 0, '202101': -18.347339896689995}
Essendon {'Initial': 0, '2020F4': 0, '202101': -3.527830929879997}
Fremantle {'Initial': 0, '2020F4': 0, '202101': -19.263733312830006}
Geelong {'Initial': 0, '2020F4': 0, '202101': 0}
Gold Coast {'Initial': 0, '2020F4': 0, '202101': -18.943709645935}
Greater Western Sydney {'Initial': 0, '2020F4': 0, '202101': 0.9166142571470033}
Hawthorn {'Initial': 0, '2020F4': 0, '202101': -6.472169070120003}
Melbourne {'Initial': 0, '2020F4': 0, '202101': 9.263733312830006}
North Melbourne {'Initial': 0, '2020F4': 0, '202101': -18.929405505107}
Port Adelaide {'Initial': 0, '2020F4': 0, '202101': 8.929405505106999}
Richmond {'Initial': 0, '2020F4': 0, '202101': 13.770569458980034}
St Kilda {'Initial': 0, '2020F4': 0, '202101': -10.9166142571

In [33]:
round_dict['202101']['202101_Richmond_Carlton'].get_match_stats()

,Away_Score,Home_Score,Away_xScore,Home_xScore,Away_exp_vaep_value,Home_exp_vaep_value,Away_exp_offensive_value,Home_exp_offensive_value,Away_exp_defensive_value,Home_exp_defensive_value
Match_ID,,,,,,,,,,
202101_Richmond_Carlton,78.0,104.0,84.197,122.7982,81.8075,119.3487,83.3134,124.6047,-1.5059,-5.256


Player Updates

In [ ]:
def update_player_ratings(Match):
    
    # Home Players Update
    home_players = Match.get_home_players()
    home_match_squad = Match.get_home_match_squad()
    home_not_match_squad = set(list(home_players.keys())) - set(home_match_squad)

    for player in home_match_squad:
        player_rating_calculator = PlayerRatingCalculator(home_players[player])
        player_rating_calculator.update_player_rating(Match)
        
    for player in home_not_match_squad:
        home_players[player].add_rating(round_id, 0)
    
    ## Away Players Update
    away_players = Match.get_away_players()
    away_match_squad = Match.get_away_match_squad()
    away_not_match_squad = set(list(away_players.keys())) - set(away_match_squad)

    for player in away_match_squad:
        player_rating_calculator = PlayerRatingCalculator(away_players[player])
        player_rating_calculator.update_player_rating(Match)
        
    for player in away_not_match_squad:
        away_players[player].add_rating(round_id, 0)


In [ ]:
for match_id in round_dict[round_id].keys():
    update_player_ratings(round_dict[round_id][match_id])

In [ ]:
for player in team_dict['Carlton'].players.keys():
    print(player, team_dict['Carlton'].players[player].rating)

## 202102

In [ ]:
round_id = "202102"

In [ ]:
round_dict = create_round_matches(player_stats, round_dict, round_id)
round_dict[round_id]

Match Projections

In [ ]:
round_id = '202102'
match_id = '202102_Carlton_Collingwood'
CarltonCollingwood202102 = round_dict[round_id][match_id]

carlton_player_ratings = team_dict['Carlton'].get_player_ratings()
carlton_squad = list(carlton_player_ratings.sort_values('202101', ascending = False).head(23).index)

collingwood_player_ratings = team_dict['Collingwood'].get_player_ratings()
collingwood_squad = list(collingwood_player_ratings.sort_values('202101', ascending = False).head(23).index)

In [ ]:
projector = Projector(Match=CarltonCollingwood202102,
                      home_squad = carlton_squad,
                      away_squad = collingwood_squad)

In [ ]:
projector.get_home_rating(), projector.get_away_rating(), projector.get_home_advantage()

In [ ]:
projector.get_margin()

In [ ]:
projector.home_prob, projector.draw_prob, projector.away_prob

In [ ]:
def update_team_ratings(Match, team_dict):
    
    home_squad_list = Match.get_home_match_squad()
    away_squad_list = Match.get_away_match_squad()
    match_projector = Projector(Match, home_squad_list, away_squad_list)
    
    home_team = Match.home_team.get_team_name()
    away_team = Match.away_team.get_team_name()
    
    home_rating_calculator = TeamRatingCalculator(team_dict[home_team])
    home_rating_calculator.update_team_rating(Match)
    
    away_rating_calculator = TeamRatingCalculator(team_dict[away_team])
    away_rating_calculator.update_team_rating(Match)

In [ ]:
# for match_id in round_dict[round_id].keys():
#     match_projector = Projector(Match=round_dict[round_id][match_id],
#                                 home_squad = carlton_squad,
#                                 away_squad = collingwood_squad)
#     update_team_ratings(round_dict[round_id][match_id])

In [ ]:
# for team in team_dict.keys():
#     print(team, team_dict[team].rating)

## 202103

End of Season